In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
companies = pd.read_csv('./merged/new_merged_companies.csv')  # Replace 'your_data.csv' with the path to your data file


In [7]:
# companies['founded_at'] = pd.to_datetime(companies['founded_at'])

missing_founded_at_count = companies['founded_at'].isnull().sum()

missing_founded_at_count
# head()

105326

In [3]:
new_aq1000 = pd.read_csv('./merged/new_acquisitions.csv', nrows=1000) 
new_fo1000 = pd.read_csv('./merged/new_financial_org.csv', nrows=1000) 
new_fr1000 = pd.read_csv('./merged/new_funding_rounds.csv', nrows=1000) 
new_if1000 = pd.read_csv('./merged/new_investments_funds.csv', nrows=1000) 
new_rp1000 = pd.read_csv('./merged/new_relationships_people.csv', nrows=1000) 

new_aq1000.to_csv('./merged/new_aq1000.csv', index=False)
new_fo1000.to_csv('./merged/new_fo1000.csv', index=False)
new_fr1000.to_csv('./merged/new_fr1000.csv', index=False)
new_if1000.to_csv('./merged/new_if1000.csv', index=False)
new_rp1000.to_csv('./merged/new_rp1000.csv', index=False)

In [11]:
# merged_comp1000 = pd.read_csv('./merged/new_merged_companies.csv', nrows=1000) # Replace 'your_data.csv' with the path to your data file

# merged_comp1000.to_csv('./merged/merged_comp1000.csv', index=False)

companies.to_csv('./merged/aged_comp2008.csv', index=False)

In [9]:


missing_founded_at_count = companies['founded_at'].isnull().sum() & companies['closed_at'].isnull().sum()

missing_founded_at_count


37132

In [5]:
new_aq = pd.read_csv('./merged/new_acquisitions.csv')

new_aq['acquired_at'] = pd.to_datetime(new_aq['acquired_at'], errors='coerce')

acquisition_dates = pd.Series(new_aq.acquired_at.values, index=new_aq.acquired_object_id).to_dict()

# Iterate through the main dataset and update 'closed_at' where necessary
for index, row in companies.iterrows():
    # Check if the company is marked as 'acquired' and 'closed_at' is missing
    if row['status'] == 'acquired' and pd.isnull(row['closed_at']):
        # Look up the 'acquired_at' date using 'id'
        acquired_at = acquisition_dates.get(row['id'])
        # If an 'acquired_at' date is found, update 'closed_at'
        if acquired_at:
            companies.at[index, 'closed_at'] = acquired_at

# Verify the updates by displaying the first few rows of the dataframe
companies.head()

,id,entity_type,name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,...,num_of_fin_org_financed,fin_org_financed,funded_object_id_y,num_of_person_financed,person_financed,funded_object_id,num_of_companies_financed,startup_financed,log_fund_tot,tot_fin
0,c:1,Company,Wetpaint,web,operating,2005-10-17,NaN,1,1,1,...,23.0,1,NaN,NaN,0,c:1,23.0,1,17.49812,2
1,c:10,Company,Flektor,games_video,acquired,NaN,2007-05-30 00:00:00,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0
2,c:100,Company,There,games_video,acquired,NaN,2005-05-29 00:00:00,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0
3,c:10000,Company,MYWEBBO,network_hosting,operating,2008-07-26,NaN,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0
4,c:10001,Company,THE Movie Streamer,games_video,operating,2008-07-26,NaN,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0


In [17]:
companies.head()

,id,entity_type,name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,...,num_of_fin_org_financed,fin_org_financed,funded_object_id_y,num_of_person_financed,person_financed,funded_object_id,num_of_companies_financed,startup_financed,log_fund_tot,tot_fin
0,c:1,Company,Wetpaint,web,operating,2005-10-17 00:00:00,NaT,1,1,1,...,23.0,1,NaN,NaN,0,c:1,23.0,1,17.49812,2
1,c:10,Company,Flektor,games_video,acquired,2007-05-31 21:11:51,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0
2,c:100,Company,There,games_video,acquired,2007-08-06 23:52:45,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0
3,c:10000,Company,MYWEBBO,network_hosting,operating,2008-07-26 00:00:00,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0
4,c:10001,Company,THE Movie Streamer,games_video,operating,2008-07-26 00:00:00,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.00000,0


In [6]:
# Count closed companies without a closed_at date
closed_no_closed_at = companies[(companies['status'] == 'closed') & (companies['closed_at'].isnull())].shape[0]

# Count acquired companies without a closed_at date
acquired_no_closed_at = companies[(companies['status'] == 'acquired') & (companies['closed_at'].isnull())].shape[0]

print("Closed companies without a closed_at date:", closed_no_closed_at)
print("Acquired companies without a closed_at date:", acquired_no_closed_at)


Closed companies without a closed_at date: 59761
Acquired companies without a closed_at date: 28


In [17]:
date_columns = ['founded_at', 'closed_at', 'last_investment_at', 'last_funding_at', 'updated_at', 'first_investment_at', 'first_funding_at', 'created_at']
for col in date_columns:
    companies[col] = pd.to_datetime(companies[col], errors='coerce')

for index, row in companies.iterrows():
    if pd.notnull(row['founded_at']):  # If 'founded_at' already has a value, check if it needs updating
        earliest_known_activity = min(row[['first_investment_at', 'first_funding_at', 'created_at']].dropna(), default=pd.NaT)
        # Update 'founded_at' only if the new date is earlier and not later than 'closed_at'
        if not pd.isnull(earliest_known_activity) and earliest_known_activity < row['founded_at']:
            if pd.isnull(row['closed_at']) or earliest_known_activity <= row['closed_at']:
                companies.at[index, 'founded_at'] = earliest_known_activity

In [8]:
current_year = pd.Timestamp(companies.founded_at.max()).year
companies['company_age'] = current_year - companies['founded_at'].dt.year

companies.company_age.describe()

count    196554.000000
mean          4.939431
std           7.483876
min           0.000000
25%           1.000000
50%           3.000000
75%           6.000000
max         113.000000
Name: company_age, dtype: float64

In [6]:
data = pd.read_csv('./merged/new_merged_companies.csv')  # Replace 'your_data.csv' with the path to your data file

# Feature Selection
# Selecting relevant columns
relevant_columns = ['category_code', 'status', 'founded_at']  # Include relevant columns based on your data
data = data[relevant_columns]

# Feature Creation
# Example: data['age_of_company'] = 2024 - pd.to_datetime(data['founded_at']).dt.year

# Feature Transformation
# Scaling numerical features
scaler = StandardScaler()
data[['funding_total_usd', 'investment_rounds']] = scaler.fit_transform(data[['funding_total_usd', 'investment_rounds']])

# Feature Encoding
encoder = OneHotEncoder(sparse=False)
encoded_columns = encoder.fit_transform(data[['category_code', 'status']])
data = data.join(pd.DataFrame(encoded_columns, columns=encoder.get_feature_names(['category_code', 'status'])))

# Target Variable Creation
Example: data['successful'] = data['status'].apply(lambda x: 1 if x in ['ipo', 'acquired'] else 0)

# Correlation Analysis
plt.figure(figsize=(12, 10))
sns.heatmap(data.corr(), annot=True, fmt=".2f")
plt.show()

# Machine Learning Model (Random Forest as an example)
X = data.drop('successful', axis=1)  # Features
y = data['successful']  # Target

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluating the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

KeyError: "None of [Index(['funding_total_usd', 'investment_rounds'], dtype='object')] are in the [columns]"